# Better way 37. 스레드를 블로킹 I/O용으로 사용하고 병렬화용으로는 사용하지 말자

* CPython
    * 파이썬의 표준 구현
        * CPython is the reference implementation of the Python programming language. Written in C and Python, CPython is the default and most widely-used implementation of the language. (wikipedia)
    * CPython 의 파이썬 프로그램의 실행 단계
        1. 소스 텍스트를 바이트코드 bytecode 로 파싱하고 컴파일한다.
        2. 스택 기반 인터프리터로 바이트코드를 실행한다.
    * 바이트코드 인터프리터
        * 파이썬 프로그램이 실행되는 동안 지속되고, ***일관성 있는 상태*** 를 유지한다.

* GIL: Global Interpreter Lock 전역 인터프리터 잠금
    * 바이트코드 인터프리터가 일관성을 유지하는 방법
        * 인터프리터에 뮤텍스 락 (Mutex lock; 상호 배제 잠금 Mutual exclusive lock) 을 건다. 
    * CPython 이 선점형 멀티스레딩의 영향을 받지 않게 해준다.
* 선점형 멀티스레딩 preemptive multithreading
    * 한 스레드가 다른 스레드를 인터럽트(차단)해서 프로그램의 제어를 얻는 것
    * 인터럽트가 예상치 못한 시간에 일어나면 인터프리터 상태가 망가짐
    * GIL 이 이런 인터럽트를 막아주며 모든 바이트코드 명령어가 CPython 구현과 C 확장 모듈에서 올바르게 동작함을 보장함

* GIL 의 부작용
    * C++ 이나 자바는 멀티스레드 = 멀티 CPU 코어
    * GIL 은 멀티스레드 = 한번에 한 스레드 = 한 CPU 코어

Example) 연산 집약적인 작업: 인수 분해 알고리즘

In [1]:
def factorize(number):
    for i in range(1, number + 1):
        if number % i == 0:
            yield i

In [2]:
from time import time, sleep

numbers = [2139079, 1214759, 1516637, 1752285]

start = time()

for number in numbers:
    print(number, list(factorize(number)))
    
end = time()
print('Took %.3f seconds' % (end - start))

2139079 [1, 101, 21179, 2139079]
1214759 [1, 7, 13, 49, 91, 637, 1907, 13349, 24791, 93443, 173537, 1214759]
1516637 [1, 19, 79823, 1516637]
1752285 [1, 3, 5, 15, 116819, 350457, 584095, 1752285]
Took 0.571 seconds


* 다른 언어에서는 이런 경우 멀티스레드를 사용할 수 있지만, 파이썬에서는...

In [3]:
from threading import Thread

class FactorizeThread(Thread):
    def __init__(self, number):
        super().__init__()
        self.number = number
        
    def run(self):
        self.factors = list(factorize(self.number))

In [4]:
start = time()

threads = []

for number in numbers:
    thread = FactorizeThread(number)
    thread.start()  # thread 의 실행
    threads.append(thread)
    
for thread in threads:
    thread.join()  # thread 가 끝날때까지 기다림
    print(thread.number, thread.factors)  # 추가한 코드
    
end = time()
print('Took %.3f seconds' % (end - start))

2139079 [1, 101, 21179, 2139079]
1214759 [1, 7, 13, 49, 91, 637, 1907, 13349, 24791, 93443, 173537, 1214759]
1516637 [1, 19, 79823, 1516637]
1752285 [1, 3, 5, 15, 116819, 350457, 584095, 1752285]
Took 0.582 seconds


* ... 순서대로 인수 분해할 때 보다 많은 시간이 걸렸다.
    * 멀티 코어에서 병렬적으로 실행되었다면 1/4 정도의 시간을 기대할 수 있음
    * 듀얼 코어라면 1/2 정도
* 즉, GIL, 표준 `threading.Thread` 클래스에서는 멀티코어로 동작하지 않음
    * Better way 41. 진정한 병렬성을 실현하려면 concurrent.futures 를 고려하자

* 멀티코어로 동작하지 못할거면 왜 `Thread` 를 구현해놨는가?
    1. **멀티스레드를 이용하면 프로그램이 동시에 여러 작업을 하는 것처럼 보이게 만들기가 유용하다.**
        * 동시에 동작하는 태스크를 관리하는 코드를 구현하기는 어려운 법임
            * Better way 40. 많은 함수를 동시에 실행하려면 코루틴을 고려하자
        * 스레드를 사용하면 함수를 마치 병렬로 실행하는 것처럼 해 주는 일을 파이썬에게 맡길 수 있다.
            * CPython 은 파이썬 스레드가 어느정도 공평하게 실행됨을 보장함
    2. **특정 유형의 시스템 호출을 수행할 때 일어나는 블로킹 I/O 를 다룰때 유용하다.**
        * 시스템 호출 system call: 프로그램이 외부 환경과 상호 작용하도록 운영체제에 요청하는 것
            * 예) 파일 읽기/쓰기, 네트워크 커뮤니케이션, 디스플레이 같은 외부 장치와의 통신 등
        * 운영체제가 이런 요청에 응답하는 데 드는 시간을 프로그램과 분리하는 데 스레드를 사용할 수 있다.

* 원격 제어가 가능한 헬리콥터에 직렬 포트로 신호 보내기
    * 느린 시스템 호출 (`select`)  이 담당함

In [5]:
import select

def slow_systemcall():
    # mock system call
    select.select([], [], [], 0.1)

In [6]:
start = time()

for _ in range(5):
    slow_systemcall()
    
end = time()
print('Took %.3f seconds' % (end - start))

Took 0.509 seconds


* 이렇게 구현하면 slow_systemcall 이 실행되는 동안 프로그램이 다른 일을 할 수 없다.
    * 신호를 헬리콥터에 보내는 동안 헬리콥터의 다음 이동을 계산해야 하는데, 할 수 없게 됨 --> 헬리콥터가 충돌하는 끔찍한 일이 벌어질수도!
* 블로킹 I/O 를 사용하는 동안 동시에 연산도 해야함
    * 스레드를 사용하자

In [7]:
import random

def compute_helicopter_location(index):
    # ...
    return index + random.random()

In [8]:
start = time()
threads = []

for _ in range(5):
    thread = Thread(target=slow_systemcall)
    thread.start()
    threads.append(thread)
    
for i in range(5):
    thread.join()
    
end = time()
print('Took %.3f seconds' % (end - start))

Took 0.106 seconds


* 이 경우에는 스레드를 사용하면 시간이 1/5 가량
    * 파이썬 코드는 병렬 X, 시스템 호출은 병렬 O
    * 파이썬 스레드가 시스템 호출을 하기 전 GIL 을 풀고 시스템 호출의 작업이 끝나는 대로 GIL 을 다시 얻기 때문

* 스레드 외에도 `asyncio` 처럼 블로킹 I/O 를 다루는 다양한 수단이 있고 여러 장점들이 있지만, 그런 옵션을 선택하면 코드를 재작성하는 추가작업이 필요함
    * Better way 40. 많은 함수를 동시에 실행하려면 코루틴을 고려하자
* 스레드 = 프로그램 수정을 최소화 하면서 블로킹 I/O 를 수행하는 가장 간단한 방법

### 핵심 정리
* 파이썬 스레드는 전역 인터프리터 잠금 (GIL, Global Interpreter Lock) 때문에 여러 CPU 코어에서 병렬로 바이트코드를 실행할 수 없다.
* GIL 에도 불구하고 파이썬 스레드는 동시에 여러 작업을 하는 것처럼 보여주기 쉽게 해 주므로 여전히 유용하다.
* 여러 시스템 호출을 병렬로 수행하려면 파이썬 스레드를 사용하자. 이렇게 하면 계산을 하면서도 블로킹 I/O 를 수행할 수 있다.